## Butler Access to Intermediate Data Products used for Verification Analyses

This is a simple example notebook to demonstrate how to access the intermediate data products with the Gen-3 butler

Verified Stack Release: w_2020_16

In [ ]:
import os
import numpy as np
import pylab
import astropy.units as u
%matplotlib widget

# Stack imports
import lsst.daf.butler as dafButler

In [ ]:
# Which version of the Stack am I using?
!eups list -s | grep lsst_distrib

In [ ]:
repo = '/project/krughoff/repos/ci_hsc_gen3'

config = os.path.join(repo,'DATA','butler.yaml')


try: butler = dafButler.Butler(config=config)
except ValueError as e: print(e)

butler = dafButler.Butler(config=config)
registry = butler.registry

In [ ]:
for c in registry.queryCollections():
    print(c)

In [ ]:
for x in registry.queryDatasetTypes():
    print(x)

## After Running the Pipeline

Let's take a look at the datasets created when running the pipeline.

In [ ]:
# Yes, you really do need to specify the collections argument
src_refs = list(registry.queryDatasets('src', collections='matchedTest', band='r'))
matched_catalog_refs = list(registry.queryDatasets('matchedCatalog', collections='matchedTest', band='r'))
pa1_refs = list(registry.queryDatasets('PA1', collections='matchedTest', band='r'))
summary_pa1_refs = list(registry.queryDatasets('meanPA1', collections='matchedTest', band='r'))
print('Found %i matched catalogs meeting the selection criteria'%(len(matched_catalog_refs)))

In [ ]:
# A variant
#dataIds = list(registry.queryDimensions(["exposure", "detector"], datasets=["calexp"], collections="shared/ci_hsc_output"))
#calexp = butler.get('calexp', dataId=dataIds[0], collections="shared/ci_hsc_output")

In [ ]:
pylab.figure()
markers = ['.', 'v', '^', '<', '>', '1', '2', '3', '4']
i = 0
for ref in matched_catalog_refs:
    matched_catalog = butler.getDirect(ref)
    if len(matched_catalog) > 0:
        pylab.scatter(np.degrees(matched_catalog['coord_ra']), np.degrees(matched_catalog['coord_dec']), 
                      edgecolor='none', marker=markers[i]) 
        i += 1
pylab.xlim(pylab.xlim()[::-1])
pylab.xlabel('RA (deg)')
pylab.ylabel('Dec (deg)')

Next, access the measured PA1 values from individual patches

In [ ]:
pa1_measurements = [butler.getDirect(ref).quantity for ref in pa1_refs]
for ref, measurement in zip(pa1_refs, pa1_measurements):
    print('Patch %3i:%10.2f %s'%(ref.dataId['patch'], 
                                 measurement.value, 
                                 measurement.unit))

pa1_measurements = u.Quantity(pa1_measurements)
    
pylab.figure()
pylab.hist(pa1_measurements.value[~np.isnan(pa1_measurements)])
pylab.xlabel('PA1 (%s)'%(pa1_measurements.unit))
pylab.ylabel('Counts')

We can also access the PA1 summary statistic

In [ ]:
butler.getDirect(summary_pa1_refs[0])